In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import *

from pyspark.ml.feature import StopWordsRemover
from pyspark import SparkConf, SparkContext
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from itertools import islice
from pyspark.mllib.feature import Word2Vec

from pyspark.sql import SparkSession

from pyspark.sql import SQLContext, Row
from pyspark.ml.feature import CountVectorizer
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.ml.linalg import Vector, Vectors, VectorUDT,SparseVector

In [3]:
from pyspark.sql import SQLContext, Row
from pyspark.ml.feature import CountVectorizer
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vector, Vectors

In [4]:
import numpy as np

In [5]:
sc = pyspark.SparkContext.getOrCreate()

In [6]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [7]:
sc.stop()

In [8]:
num_rows_to_show = 20
text_file = 'data/listings.csv'

In [9]:
sc = SparkContext() # pyspark.SparkContext.getOrCreate()
spark = SparkSession(sc) # SparkSession.builder.appName('ops').getOrCreate()

In [12]:
df = spark.read.csv(text_file, inferSchema=True, header=True)
corpus = df.select("id", "name").dropna(subset="name")

In [13]:
tokenizer = Tokenizer(inputCol="name", outputCol="words")
docDF = tokenizer.transform(corpus)

In [14]:
Vector = CountVectorizer(inputCol="words", outputCol="vectors", minDF=2.0)
model = Vector.fit(docDF)
result = model.transform(docDF)

In [17]:
result.show(10, False)

+-----+-------------------------------------------------+---------------------------------------------------------+--------------------------------------------------------------------+
|id   |name                                             |words                                                    |vectors                                                             |
+-----+-------------------------------------------------+---------------------------------------------------------+--------------------------------------------------------------------+
|2818 |Quiet Garden View Room & Super Fast WiFi         |[quiet, garden, view, room, &, super, fast, wifi]        |(2674,[8,9,17,31,51,141,237,1189],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|20168|100%Centre-Studio 1 Private Floor/Bathroom       |[100%centre-studio, 1, private, floor/bathroom]          |(2674,[20,103,2583,2615],[1.0,1.0,1.0,1.0])                         |
|25428|Lovely apt in City Centre (Jordaan)              |[lovely, apt, in, 

### Here

In [18]:
result.select("vectors").show(10, False)

+--------------------------------------------------------------------+
|vectors                                                             |
+--------------------------------------------------------------------+
|(2674,[8,9,17,31,51,141,237,1189],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|(2674,[20,103,2583,2615],[1.0,1.0,1.0,1.0])                         |
|(2674,[1,6,11,25,32,489],[1.0,1.0,1.0,1.0,1.0,1.0])                 |
|(2674,[1,22,52,67,111,134,802],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])       |
|(2674,[6,8,10,11,15,301],[1.0,1.0,1.0,1.0,1.0,1.0])                 |
|(2674,[8,57,105],[1.0,1.0,1.0])                                     |
|(2674,[8,57,308],[1.0,1.0,1.0])                                     |
|(2674,[0,45,53,85,1168],[1.0,1.0,1.0,1.0,1.0])                      |
|(2674,[1,4,7,42,48,51,84],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])            |
|(2674,[0,2,14,243],[1.0,1.0,1.0,1.0])                               |
+--------------------------------------------------------------------+
only s

In [127]:
from pyspark.sql import Row

# Function to update dataframe row with a rownumber
def create_rownum(ziprow):
    row, index=ziprow
    row=row.asDict()
    row['rownum2']= row['vectors'].values
    return(Row(**row))

# First create a rownumber then add to dataframe
result2.rdd.zipWithIndex().map(create_rownum).toDF().show()

TypeError: not supported type: <class 'numpy.ndarray'>

In [ ]:
from pyspark.sql.functions import col, create_map, lit
from itertools import chain

mapping_expr = create_map([lit(x) for x in chain(*mapping.items())])

result.withColumn("value", mapping_expr.getItem(col("vectors")))


In [117]:
from pyspark.sql.functions import lit

In [140]:
def tt(a):
    return a.toArray()

In [141]:
result2 = result.withColumn("n", lit(tt(result.vectors)))
result2.show()

TypeError: 'Column' object is not callable

In [29]:
for f in result.collect(): 
        print (f.vectors.toArray)


<bound method SparseVector.toArray of SparseVector(2674, {8: 1.0, 9: 1.0, 17: 1.0, 31: 1.0, 51: 1.0, 141: 1.0, 237: 1.0, 1189: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {20: 1.0, 103: 1.0, 2583: 1.0, 2615: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 6: 1.0, 11: 1.0, 25: 1.0, 32: 1.0, 489: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 22: 1.0, 52: 1.0, 67: 1.0, 111: 1.0, 134: 1.0, 802: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {6: 1.0, 8: 1.0, 10: 1.0, 11: 1.0, 15: 1.0, 301: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {8: 1.0, 57: 1.0, 105: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {8: 1.0, 57: 1.0, 308: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 45: 1.0, 53: 1.0, 85: 1.0, 1168: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 4: 1.0, 7: 1.0, 42: 1.0, 48: 1.0, 51: 1.0, 84: 1.0})>
<bound method 

<bound method SparseVector.toArray of SparseVector(2674, {28: 2.0, 69: 2.0, 206: 1.0, 632: 1.0, 808: 2.0, 1574: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 24: 1.0, 44: 1.0, 917: 1.0, 1020: 1.0, 1053: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 21: 1.0, 596: 1.0, 2446: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 20: 1.0, 22: 1.0, 28: 1.0, 41: 1.0, 224: 1.0, 380: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 1: 1.0, 4: 1.0, 52: 1.0, 376: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {2: 1.0, 6: 1.0, 11: 1.0, 44: 1.0, 275: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 3: 1.0, 15: 1.0, 77: 1.0, 83: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 6: 1.0, 16: 1.0, 250: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {12: 1.0, 20: 1.0, 21: 1.0, 37: 2.0, 45: 1.0, 62: 1.0, 83: 1.0, 173: 2.0})>
<bo

<bound method SparseVector.toArray of SparseVector(2674, {3: 1.0, 25: 1.0, 80: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 20: 1.0, 217: 1.0, 254: 1.0, 2555: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {9: 1.0, 168: 1.0, 302: 1.0, 406: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {3: 1.0, 17: 1.0, 29: 1.0, 43: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 19: 1.0, 24: 1.0, 29: 1.0, 241: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 4: 1.0, 13: 1.0, 14: 1.0, 15: 1.0, 38: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {5: 1.0, 27: 1.0, 41: 1.0, 80: 1.0, 83: 1.0, 163: 1.0, 523: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 6: 1.0, 382: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {3: 1.0, 12: 1.0, 25: 1.0, 77: 1.0, 83: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 4: 2.0, 6: 1.0, 

<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 3: 1.0, 43: 1.0, 56: 1.0, 93: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 2: 1.0, 17: 1.0, 27: 1.0, 53: 1.0, 66: 1.0, 1388: 1.0, 1508: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 15: 1.0, 70: 1.0, 256: 1.0, 363: 1.0, 730: 1.0, 1371: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 3: 1.0, 15: 1.0, 28: 2.0, 35: 1.0, 56: 1.0, 313: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 45: 1.0, 53: 1.0, 85: 1.0, 509: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 3: 1.0, 17: 1.0, 54: 1.0, 81: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 8: 1.0, 30: 1.0, 132: 1.0, 1527: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {3: 1.0, 8: 1.0, 20: 1.0, 483: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 2: 1.0, 27: 1.0, 41: 1.0})>
<bound meth

<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 1: 1.0, 6: 1.0, 25: 1.0, 119: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {3: 1.0, 12: 1.0, 17: 1.0, 35: 1.0, 110: 1.0, 816: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 9: 1.0, 32: 1.0, 66: 1.0, 100: 1.0, 375: 1.0, 773: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {32: 1.0, 126: 1.0, 138: 1.0, 142: 1.0, 158: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {32: 1.0, 37: 1.0, 45: 1.0, 62: 1.0, 73: 1.0, 1871: 1.0, 2212: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {6: 1.0, 14: 1.0, 27: 1.0, 32: 1.0, 41: 1.0, 298: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 9: 1.0, 15: 1.0, 40: 1.0, 47: 1.0, 576: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {26: 1.0, 27: 1.0, 35: 1.0, 44: 1.0, 76: 1.0, 122: 1.0})>
<bound metho

...

<bound method SparseVector.toArray of SparseVector(2674, {3: 1.0, 17: 1.0, 19: 1.0, 34: 1.0, 60: 1.0, 354: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 6: 1.0, 8: 1.0, 11: 1.0, 16: 1.0, 520: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 1: 1.0, 2: 1.0, 19: 1.0, 23: 1.0, 93: 1.0, 162: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {13: 1.0, 25: 1.0, 32: 1.0, 36: 1.0, 38: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 1: 1.0, 2: 1.0, 19: 1.0, 23: 1.0, 150: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {2: 1.0, 8: 1.0, 19: 1.0, 32: 1.0, 44: 2.0, 117: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {4: 1.0, 11: 1.0, 13: 1.0, 29: 1.0, 38: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 2: 1.0, 29: 1.0, 41: 1.0, 95: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 1: 1.0, 33: 1.0, 96: 1.0, 182: 1.0, 208: 1.0, 965:

<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 10: 1.0, 88: 1.0, 108: 1.0, 113: 1.0, 283: 1.0, 751: 1.0, 1052: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 7: 1.0, 23: 1.0, 43: 1.0, 372: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {901: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 9: 1.0, 25: 1.0, 32: 1.0, 43: 1.0, 45: 1.0, 62: 1.0, 66: 1.0, 93: 1.0, 94: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 3: 1.0, 6: 1.0, 14: 1.0, 29: 1.0, 52: 1.0, 293: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 5: 1.0, 8: 1.0, 25: 1.0, 28: 1.0, 32: 1.0, 105: 1.0, 205: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 1: 1.0, 3: 1.0, 5: 1.0, 43: 1.0, 56: 1.0, 1199: 1.0, 1736: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 10: 1.0, 88: 1.0, 108: 1.0, 113: 1.0, 283: 1.0, 751: 1.0, 1052: 1.0})>
<bound method Sparse

<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 1: 1.0, 2: 1.0, 16: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {8: 1.0, 10: 1.0, 14: 1.0, 20: 1.0, 92: 1.0, 123: 1.0, 1076: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {6: 1.0, 11: 1.0, 308: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 2: 1.0, 49: 1.0, 54: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {34: 1.0, 63: 1.0, 202: 1.0, 478: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {5: 1.0, 7: 1.0, 32: 1.0, 33: 1.0, 37: 2.0, 95: 1.0, 166: 1.0, 800: 1.0, 1200: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 1: 1.0, 5: 1.0, 25: 1.0, 131: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {21: 1.0, 30: 1.0, 825: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 21: 1.0, 25: 1.0, 46: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 5: 1.0, 

<bound method SparseVector.toArray of SparseVector(2674, {15: 1.0, 20: 1.0, 50: 1.0, 133: 1.0, 161: 1.0, 609: 1.0, 675: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {143: 1.0, 153: 1.0, 290: 1.0, 1089: 1.0, 2468: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {3: 1.0, 20: 1.0, 41: 1.0, 69: 2.0, 77: 1.0, 92: 1.0, 344: 1.0, 2628: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {12: 1.0, 78: 1.0, 86: 1.0, 171: 1.0, 865: 1.0, 1042: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 3: 1.0, 62: 1.0, 72: 1.0, 167: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 4: 1.0, 50: 1.0, 637: 1.0, 2090: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {2: 1.0, 384: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {159: 1.0, 197: 1.0, 487: 1.0, 648: 1.0, 1628: 1.0, 2284: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 2: 1.0, 15: 1.0, 41: 1.0, 113: 1.

<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 6: 1.0, 13: 1.0, 14: 1.0, 19: 1.0, 58: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 1: 1.0, 41: 1.0, 70: 1.0, 103: 1.0, 111: 1.0, 333: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 1: 1.0, 31: 1.0, 41: 1.0, 70: 1.0, 103: 1.0, 160: 1.0, 333: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 1: 1.0, 27: 1.0, 41: 1.0, 70: 1.0, 333: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {1: 1.0, 4: 1.0, 6: 1.0, 14: 1.0, 67: 1.0, 298: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 1: 1.0, 27: 1.0, 41: 1.0, 70: 1.0, 333: 1.0, 2378: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0, 1: 1.0, 27: 1.0, 41: 1.0, 70: 1.0, 74: 1.0, 333: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {3: 1.0, 26: 1.0, 31: 1.0, 796: 1.0})>
<bound method SparseVector.toArray of SparseVector(2674, {0: 1.0

In [103]:
import pyspark.sql.functions as f

df2 = sqlCtx.createDataFrame([(0, ["a", "b", "c", "x", "y"]),
                            (1, ["a", "b", "b", "c", "a"])],
                            ["label", "raw"])

combined_df = (
    df2.select(f.explode('raw').alias('col'))
      .select(f.collect_list('col').alias('raw'))
)
combined_df.show(truncate=False)

NameError: name 'sqlCtx' is not defined

In [104]:
counts = result.select(f.explode('raw').alias('col')).groupBy('col').count().collect()
print(counts)

AnalysisException: "cannot resolve '`raw`' given input columns: [id, name, words, vectors];;\n'Project [explode('raw) AS col#483]\n+- Project [id#82, name#83, words#319, UDF(words#319) AS vectors#400]\n   +- Project [id#82, name#83, UDF(name#83) AS words#319]\n      +- Filter AtLeastNNulls(n, name#83)\n         +- Project [id#82, name#83]\n            +- Relation[id#82,name#83,host_id#84,host_name#85,neighbourhood_group#86,neighbourhood#87,latitude#88,longitude#89,room_type#90,price#91,minimum_nights#92,number_of_reviews#93,last_review#94,reviews_per_month#95,calculated_host_listings_count#96,availability_365#97] csv\n"

+-----+-------------------------------------------------+---------------------------------------------------------+--------------------------------------------------------------------+
|id   |name                                             |words                                                    |vectors                                                             |
+-----+-------------------------------------------------+---------------------------------------------------------+--------------------------------------------------------------------+
|2818 |Quiet Garden View Room & Super Fast WiFi         |[quiet, garden, view, room, &, super, fast, wifi]        |(2674,[8,9,17,31,51,141,237,1189],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|20168|100%Centre-Studio 1 Private Floor/Bathroom       |[100%centre-studio, 1, private, floor/bathroom]          |(2674,[20,103,2583,2615],[1.0,1.0,1.0,1.0])                         |
|25428|Lovely apt in City Centre (Jordaan)              |[lovely, apt, in, 

In [92]:
result.select('vectors', 'id')

DataFrame[vectors: vector, id: string]

In [101]:
def parseVectors(line):
    print(line[0][1])
    return [int(line[1]), line[0][1]]


sparsevector = result.select('vectors', 'id').rdd.map(parseVectors)


In [102]:
sparsevector.toDF().show(10)

Py4JJavaError: An error occurred while calling o1892.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 82.0 failed 1 times, most recent failure: Lost task 0.0 in stage 82.0 (TID 85, localhost, executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for numpy.dtype)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.api.python.SerDeUtil$$anonfun$pythonToJava$1$$anonfun$apply$1.apply(SerDeUtil.scala:188)
	at org.apache.spark.api.python.SerDeUtil$$anonfun$pythonToJava$1$$anonfun$apply$1.apply(SerDeUtil.scala:187)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:435)


In [111]:
#Train the LDA model
model = LDA.train(f.vectors.toArray, k=5, seed=1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.mllib.api.python.SerDe.loads.
: net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for pyspark.ml.linalg.SparseVector)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.mllib.api.python.SerDeBase.loads(PythonMLLibAPI.scala:1318)
	at org.apache.spark.mllib.api.python.SerDe.loads(PythonMLLibAPI.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Met

In [48]:
result.show(10, False)

+-----+-------------------------------------------------+---------------------------------------------------------+--------------------------------------------------------------------+
|id   |name                                             |words                                                    |vectors                                                             |
+-----+-------------------------------------------------+---------------------------------------------------------+--------------------------------------------------------------------+
|2818 |Quiet Garden View Room & Super Fast WiFi         |[quiet, garden, view, room, &, super, fast, wifi]        |(7254,[8,9,17,31,51,141,236,1186],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|20168|100%Centre-Studio 1 Private Floor/Bathroom       |[100%centre-studio, 1, private, floor/bathroom]          |(7254,[20,103,1930,2448],[1.0,1.0,1.0,1.0])                         |
|25428|Lovely apt in City Centre (Jordaan)              |[lovely, apt, in, 

In [169]:
tokenizer = Tokenizer(inputCol="name", outputCol="raw_words")

In [170]:
wordsData = tokenizer.transform(result) #corpus

In [171]:
locale = spark._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))

StopWordsRemover.loadDefaultStopWords("english")

remover = StopWordsRemover(inputCol="raw_words", outputCol="words2")
wordsData = remover.transform(wordsData)

In [172]:
hashingTF = HashingTF(inputCol="words2", outputCol="rawFeatures", numFeatures=10000)
featurizedData = hashingTF.transform(wordsData)

In [173]:
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=1)
idfModel = idf.fit(featurizedData)

tfidf = idfModel.transform(featurizedData)

In [183]:
tfidf.select("features").show(10, False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                            |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(10000,[494,1692,1789,2659,7293,8048,8562,9263],[2.445567235227968,2.8094747144167127,3.6888794541139363,5.2799682278798405,3.247046701834897,5.983267779903803,2.46837734783679,8.083328608786376])|
|(10000,[4235,4744,8704,9743],[7.102499355774649,4.745189363091357,2.909063890908317,8.776475789346321])                                                                                             |
|(100

#### PROBLEM IS HERE

In [40]:
num_topics = 10
max_iterations = 100

lda_model = LDA.train(tfidf[['id','features']].rdd, k=num_topics, maxIterations=max_iterations)

Py4JJavaError: An error occurred while calling o607.trainLDAModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 28.0 failed 1 times, most recent failure: Lost task 0.0 in stage 28.0 (TID 27, localhost, executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for pyspark.ml.linalg.SparseVector)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.mllib.api.python.SerDeBase$$anonfun$pythonToJava$1$$anonfun$apply$2.apply(PythonMLLibAPI.scala:1350)
	at org.apache.spark.mllib.api.python.SerDeBase$$anonfun$pythonToJava$1$$anonfun$apply$2.apply(PythonMLLibAPI.scala:1349)
	at scala.collection.I

#### AND HERE

In [47]:
# data = sc.textFile(text_file)
# parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))

df = spark.read.csv(text_file, inferSchema=True, header=True)
corpusMy = df.select("id", "name").dropna(subset="name")

In [56]:
corpusMy.select("id", "name").show(10)
test = corpusMy.select("id", "name").rdd.map(lambda x: [x[1], x[0]]).collect().parralelize()
print(test)


+-----+--------------------+
|   id|                name|
+-----+--------------------+
| 2818|Quiet Garden View...|
|20168|100%Centre-Studio...|
|25428|Lovely apt in Cit...|
|27886|Romantic, stylish...|
|28658|Cosy guest room n...|
|28871|Comfortable doubl...|
|29051|Comfortable singl...|
|31080|2-story apartment...|
|38266|Nice and quiet pl...|
|41125|Amsterdam Center ...|
+-----+--------------------+
only showing top 10 rows



AttributeError: 'list' object has no attribute 'parralelize'

In [54]:
lda_model = LDA.train(test, k=num_topics, maxIterations=max_iterations)

Py4JJavaError: An error occurred while calling o1048.trainLDAModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 45.0 failed 1 times, most recent failure: Lost task 0.0 in stage 45.0 (TID 44, localhost, executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for pyspark.sql.types._create_row)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.mllib.api.python.SerDeBase$$anonfun$pythonToJava$1$$anonfun$apply$2.apply(PythonMLLibAPI.scala:1350)
	at org.apache.spark.mllib.api.python.SerDeBase$$anonfun$pythonToJava$1$$anonfun$apply$2.apply(PythonMLLibAPI.scala:1349)
	at scala.collection.I

In [43]:
#corpus = result_tfidf.select("index", "features").rdd.map(lambda x,y: [x,Vectors.fromML(y)]).cache()
corpus = result_tfidf.select("index", "features").rdd.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()
result_tfidf.select("index", "features").head()

num_topics = 10
max_iterations = 100

lda_model = LDA.train(corpus, k=num_topics, maxIterations=max_iterations)

Py4JJavaError: An error occurred while calling o757.trainLDAModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 31.0 failed 1 times, most recent failure: Lost task 0.0 in stage 31.0 (TID 30, localhost, executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for pyspark.ml.linalg.SparseVector)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.mllib.api.python.SerDeBase$$anonfun$pythonToJava$1$$anonfun$apply$2.apply(PythonMLLibAPI.scala:1350)
	at org.apache.spark.mllib.api.python.SerDeBase$$anonfun$pythonToJava$1$$anonfun$apply$2.apply(PythonMLLibAPI.scala:1349)
	at scala.collection.I

In [36]:
# topics = ldaModel.topicsMatrix()
# vocabArray = model.vocabulary

In [ ]:
# wordNumbers = 10  # number of words per topic
# topicIndices = sc.parallelize(ldaModel.describeTopics(maxTermsPerTopic = wordNumbers))

# def topic_render(topic):  # specify vector id of words to actual words
#     terms = topic[0]
#     result = []
#     for i in range(wordNumbers):
#         term = vocabArray[terms[i]]
#         result.append(term)
#     return result

# topics_final = topicIndices.map(lambda topic: topic_render(topic)).collect()

# for topic in range(len(topics_final)):
#     print ("Topic" + str(topic) + ":")
#     for term in topics_final[topic]:
#         print (term)
#     print ('\n')

### Approache 3
https://stackoverflow.com/questions/42051184/latent-dirichlet-allocation-lda-in-spark

In [23]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

# Load and parse the data
data = sc.textFile("./sample_lda_data.txt")
parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
# Index documents with unique IDs
corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()



In [24]:
a = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))

In [53]:
corpus.toDF().show(10)

+---+--------------------+
| _1|                  _2|
+---+--------------------+
|  0|[1.0,2.0,6.0,0.0,...|
|  1|[1.0,3.0,0.0,1.0,...|
|  2|[1.0,4.0,1.0,0.0,...|
|  3|[2.0,1.0,0.0,3.0,...|
|  4|[3.0,1.0,1.0,9.0,...|
|  5|[4.0,2.0,0.0,3.0,...|
|  6|[2.0,1.0,0.0,3.0,...|
|  7|[1.0,1.0,1.0,9.0,...|
|  8|[4.0,4.0,0.0,3.0,...|
|  9|[2.0,8.0,2.0,0.0,...|
+---+--------------------+
only showing top 10 rows



In [60]:
corpus.toDF().select("_2").show(10, False)

+---------------------------------------------+
|_2                                           |
+---------------------------------------------+
|[1.0,2.0,6.0,0.0,2.0,3.0,1.0,1.0,0.0,0.0,3.0]|
|[1.0,3.0,0.0,1.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0]|
|[1.0,4.0,1.0,0.0,0.0,4.0,9.0,0.0,1.0,2.0,0.0]|
|[2.0,1.0,0.0,3.0,0.0,0.0,5.0,0.0,2.0,3.0,9.0]|
|[3.0,1.0,1.0,9.0,3.0,0.0,2.0,0.0,0.0,1.0,3.0]|
|[4.0,2.0,0.0,3.0,4.0,5.0,1.0,1.0,1.0,4.0,0.0]|
|[2.0,1.0,0.0,3.0,0.0,0.0,5.0,0.0,2.0,2.0,9.0]|
|[1.0,1.0,1.0,9.0,2.0,1.0,2.0,0.0,0.0,1.0,3.0]|
|[4.0,4.0,0.0,3.0,4.0,2.0,1.0,3.0,0.0,0.0,0.0]|
|[2.0,8.0,2.0,0.0,3.0,0.0,2.0,0.0,2.0,7.0,2.0]|
+---------------------------------------------+
only showing top 10 rows



In [26]:
corpus.toDF().show(10, True)

+---+--------------------+
| _1|                  _2|
+---+--------------------+
|  0|[1.0,2.0,6.0,0.0,...|
|  1|[1.0,3.0,0.0,1.0,...|
|  2|[1.0,4.0,1.0,0.0,...|
|  3|[2.0,1.0,0.0,3.0,...|
|  4|[3.0,1.0,1.0,9.0,...|
|  5|[4.0,2.0,0.0,3.0,...|
|  6|[2.0,1.0,0.0,3.0,...|
|  7|[1.0,1.0,1.0,9.0,...|
|  8|[4.0,4.0,0.0,3.0,...|
|  9|[2.0,8.0,2.0,0.0,...|
+---+--------------------+
only showing top 10 rows



In [36]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def array_to_string(my_list):
    return '[' + ','.join([str(elem) for elem in my_list]) + ']'



In [37]:
array_to_string_udf = udf(array_to_string,StringType())



In [42]:
df = corpus.toDF().withColumn('_3',array_to_string_udf(corpus.toDF()["_2"]))

AnalysisException: 'Resolved attribute(s) _2#67 missing from _1#62L,_2#63 in operator !Project [_1#62L, _2#63, array_to_string(_2#67) AS _3#71]. Attribute(s) with the same name appear in the operation: _2. Please check if the right attribute(s) are used.;;\n!Project [_1#62L, _2#63, array_to_string(_2#67) AS _3#71]\n+- LogicalRDD [_1#62L, _2#63], false\n'

In [30]:
#corpus.toDF().write.txt('mycsv.txt')
corpus.toDF().repartition(1).write.csv("cc_out.csv", sep='|')

AnalysisException: 'CSV data source does not support struct<type:tinyint,size:int,indices:array<int>,values:array<double>> data type.;'

In [94]:
corpusMy.select("id", "name").show(10)
test = corpusMy.select("id", "name").rdd.map(lambda x: [x[1], x[0]]).collect().parralelize()
print(test)


+-----+--------------------+
|   id|                name|
+-----+--------------------+
| 2818|Quiet Garden View...|
|20168|100%Centre-Studio...|
|25428|Lovely apt in Cit...|
|27886|Romantic, stylish...|
|28658|Cosy guest room n...|
|28871|Comfortable doubl...|
|29051|Comfortable singl...|
|31080|2-story apartment...|
|38266|Nice and quiet pl...|
|41125|Amsterdam Center ...|
+-----+--------------------+
only showing top 10 rows



AttributeError: 'list' object has no attribute 'parralelize'

In [95]:
# Cluster the documents into three topics using LDA
ldaModel = LDA.train(corpus, k=3)


In [96]:

# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())
      + " words):")
topics = ldaModel.topicsMatrix()
for topic in range(3):
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        print(" " + str(topics[word][topic]))

Learned topics (as distributions over vocab of 11 words):
Topic 0:
 12.766212964867128
 5.404062581296598
 1.2080547519291418
 11.224732916708373
 7.271996259247922
 6.272972206891282
 13.592066657244391
 4.3040630266363955
 3.5758103017733465
 6.387035011756948
 14.688940666297647
Topic 1:
 5.447136988801134
 10.34828688951476
 2.4032038384806174
 19.72500381914468
 7.548150077199549
 5.91222146284111
 9.780236217644898
 1.4578817678328848
 2.677072435605984
 11.479444559843477
 10.156443464835043
Topic 2:
 7.786650046331737
 13.24765052918864
 8.388741409590242
 9.050263264146945
 10.179853663552528
 9.814806330267606
 7.627697125110712
 4.238055205530719
 1.7471172626206697
 6.133520428399574
 8.15461586886731


In [58]:
tokenizer = Tokenizer(inputCol="name", outputCol="words")
docDF = tokenizer.transform(corpus2)
Vector = CountVectorizer(inputCol="words", outputCol="vectors")
model = Vector.fit(docDF)
result = model.transform(docDF)

In [61]:
corpus = result.select("id", "vectors").rdd.map(lambda x,y: [x,Vectors.fromML(y)]).cache()

#### PROBLEM IS HERE

In [62]:
ldaModel = LDA.train(corpus, k=3,maxIterations=100,optimizer='online')

Py4JJavaError: An error occurred while calling o1392.trainLDAModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 35.0 failed 1 times, most recent failure: Lost task 0.0 in stage 35.0 (TID 33, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
TypeError: <lambda>() missing 1 required positional argument: 'y'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.